In [ ]:
from comprehensive_selection import *

from jqdata import *
import pandas as pd
# 获取当前日期和时间
current_datetime = datetime.datetime.now()
current_date = current_datetime.strftime('%Y-%m-%d')

# 获取所有交易日
trade_dates = get_trade_days(end_date=current_date, count=2)

# 获取当前时间的小时
current_hour = current_datetime.hour

# 将日期转换为字符串格式
last_trade_date = trade_dates[-1].strftime('%Y-%m-%d')
first_trade_date = trade_dates[0].strftime('%Y-%m-%d')

# 判断最后交易日是否为当天且时间是否在16:00前
if last_trade_date == current_date and current_hour < 16:
    specified_date = first_trade_date
else:
    specified_date = last_trade_date





In [ ]:
# 对每个交易日运行选股策略
for date in [specified_date]:
    print(f"\n=== {date} 的选股结果 ===")
    result = run_stock_selection(date)
    print(f"筛选的股票代码：{result}")

In [ ]:
# 从daily_bottom_finder模块导入所需的函数和类
from candle_stick_ananly import (
    get_low_and_high
)

# 计算每个交易日股票的入场价格
for date in [specified_date]:
    # 指定要分析的股票代码列表
    stock_codes = get_stocks_by_date(date)
    
    # 获取当日股票数据
    df = get_low_and_high(date, stock_codes)
    # 保存分析结果到数据库
    os.makedirs('daily_rs', exist_ok=True)
    conn = init_database()
    save_low_and_high(conn,df, date)
    conn.close()

    # 选择需要显示的列
    display_cols = ['lc', 'ld', 'lx', 'la']
    
    # 创建结果DataFrame
    result_df = pd.DataFrame(index=df.index)
    result_df.index.name = 'security'
    
    # 复制选定的列
    for col in display_cols:
        result_df[col] = df[col]
    
    # 计算目标价格
    result_df['target_price'] = result_df['la'] * 1.05
    # 设置列名
    result_df.columns = ['底C', '底D', '底X', '入场价', '目标价']
    # 格式化输出
    pd.set_option('display.float_format', lambda x: '%.2f' % x)
    
    print(result_df)